<a href="https://colab.research.google.com/github/yelallioui/Python-DataScience-Master-IA-GI/blob/main/Notebooks/04_SciPy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/yelallioui/Python-DataScience-Master-IA-GI/blob/main/Notebooks/04_SciPy.ipynb" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

# Introduction à SciPy - Calculs Scientifiques et Outils d'Analyse Avancée en Python
_Master IA-GI — Notebook 4_

Ce notebook suit la structure de votre support **SciPy** et met l'accent sur les sous-modules clés :  
`scipy.linalg`, `scipy.optimize`, `scipy.integrate`, `scipy.interpolate`, `scipy.stats`, `scipy.signal`, `scipy.fft`, `scipy.spatial`, `scipy.ndimage`.

**Objectifs d’apprentissage**
- Résoudre des problèmes classiques de **calcul scientifique** avec SciPy
- Comprendre quand utiliser **SciPy vs NumPy**
- Appliquer **optimisation**, **intégration**, **interpolation**, **statistiques**, **signal/FFT**, **spatial**, **imagerie**
- Construire un mini‑projet intégrant plusieurs briques SciPy

**Structure du cours :**
- **Chapitre 1** : Introduction à SciPy et manipulation scientifique de base
- **Chapitre 2** : Analyse scientifique et structures avancées
- **Chapitre 3** : Applications scientifiques et préparation aux projets IA

**Pré-requis** : NumPy (Notebook 3) + Matplotlib (Notebook 4)  
**Durée estimée** : 3–4h

---
# CHAPITRE 1 : Introduction à SciPy et Manipulation Scientifique de Base
---

## 1.1 Introduction à SciPy

### 📌 Rappel essentiel

- **SciPy** = Scientific Python, bibliothèque de calcul scientifique construite sur NumPy
- Fournit des fonctions optimisées pour : optimisation, statistiques, interpolation, traitement du signal, etc.
- Code majoritairement en Python, avec des parties en C pour les performances
- Projet open source créé par Travis Oliphant (créateur de NumPy)
- Code source : https://github.com/scipy/scipy

## 1.2 Installation de SciPy

In [ ]:
# Installation de SciPy (déjà installé sur Google Colab)
# Décommenter si nécessaire :
# !pip install scipy

In [ ]:
# Vérification de la version installée
import scipy
print(f"Version de SciPy : {scipy.__version__}")

In [ ]:
# Importation des bibliothèques de base
import numpy as np
import matplotlib.pyplot as plt

print("NumPy et Matplotlib importés avec succès !")

## 1.3 Constantes Scientifiques (`scipy.constants`)

### 📌 Rappel essentiel

- Module fournissant des centaines de constantes mathématiques, physiques et d'unités
- Évite les erreurs de précision lors de la saisie manuelle
- Catégories : préfixes SI, masse, longueur, temps, pression, vitesse, température, énergie, etc.
- Toutes les valeurs sont en unités SI (mètres, kilogrammes, secondes, etc.)
- Fonction `dir(constants)` pour lister toutes les constantes disponibles

In [ ]:
# Importation du module constants
from scipy import constants

# Constante mathématique : pi
print(f"π (pi) = {constants.pi}")

# Conversion : 1 litre en mètres cubes
print(f"1 litre = {constants.liter} m³")

In [ ]:
# Afficher quelques constantes disponibles
print("Exemples de constantes disponibles :")
print(dir(constants)[:20])  # Affiche les 20 premières

In [ ]:
# === PRÉFIXES MÉTRIQUES (SI) ===
# Chaque préfixe correspond à une puissance de 10

print("=== Préfixes métriques (SI) ===")
print(f"kilo  = {constants.kilo}   (10³)")
print(f"milli = {constants.milli}  (10⁻³)")
print(f"micro = {constants.micro}  (10⁻⁶)")
print(f"yotta = {constants.yotta}  (10²⁴)")

In [ ]:
# === PRÉFIXES BINAIRES (Informatique) ===
# Utilisés pour la mémoire informatique

print("=== Préfixes binaires ===")
print(f"kibi = {constants.kibi}        (2¹⁰)")
print(f"mebi = {constants.mebi}     (2²⁰)")
print(f"gibi = {constants.gibi}  (2³⁰)")

In [ ]:
# === MASSE (en kilogrammes) ===

print("=== Masse (en kg) ===")
print(f"1 gramme     = {constants.gram} kg")
print(f"1 livre      = {constants.pound} kg")
print(f"1 tonne      = {constants.metric_ton} kg")
print(f"1 u (atomic) = {constants.atomic_mass} kg")

In [ ]:
# === ANGLES (en radians) ===

print("=== Angles (en radians) ===")
print(f"1 degré    = {constants.degree} rad")
print(f"1 arcmin   = {constants.arcminute} rad")
print(f"1 arcsec   = {constants.arcsecond} rad")

In [ ]:
# === TEMPS (en secondes) ===

print("=== Temps (en secondes) ===")
print(f"1 minute = {constants.minute} s")
print(f"1 heure  = {constants.hour} s")
print(f"1 jour   = {constants.day} s")
print(f"1 semaine = {constants.week} s")
print(f"1 année  = {constants.year} s")

In [ ]:
# === LONGUEUR (en mètres) ===

print("=== Longueur (en mètres) ===")
print(f"1 inch         = {constants.inch} m")
print(f"1 foot         = {constants.foot} m")
print(f"1 mile         = {constants.mile} m")
print(f"1 mile nautique = {constants.nautical_mile} m")
print(f"1 année-lumière = {constants.light_year} m")

In [ ]:
# === PRESSION (en Pascals) ===

print("=== Pression (en Pa) ===")
print(f"1 atm  = {constants.atm} Pa")
print(f"1 bar  = {constants.bar} Pa")
print(f"1 torr = {constants.torr} Pa")
print(f"1 psi  = {constants.psi} Pa")

In [ ]:
# === VITESSE (en m/s) ===

print("=== Vitesse (en m/s) ===")
print(f"1 km/h        = {constants.kmh} m/s")
print(f"1 mph         = {constants.mph} m/s")
print(f"vitesse du son = {constants.speed_of_sound} m/s")
print(f"1 nœud        = {constants.knot} m/s")

In [ ]:
# === TEMPÉRATURE (en Kelvin) ===

print("=== Température ===")
print(f"0°C en Kelvin = {constants.zero_Celsius} K")
print(f"1°F en K⁻¹    = {constants.degree_Fahrenheit}")

In [ ]:
# === ÉNERGIE (en Joules) ===

print("=== Énergie (en J) ===")
print(f"1 eV      = {constants.electron_volt} J")
print(f"1 calorie = {constants.calorie} J")
print(f"1 BTU     = {constants.Btu} J")

In [ ]:
# === PUISSANCE (en Watts) et FORCE (en Newtons) ===

print("=== Puissance (en W) ===")
print(f"1 horsepower = {constants.horsepower} W")

print("\n=== Force (en N) ===")
print(f"1 kgf = {constants.kilogram_force} N")
print(f"1 lbf = {constants.pound_force} N")

## 1.4 Optimisation (`scipy.optimize`)

### 📌 Rappel essentiel

- **Objectif** : Trouver la meilleure solution à un problème (minimum/maximum d'une fonction)
- **Fonctions principales** :
  - `curve_fit` : Ajustement de courbe par moindres carrés
  - `root` : Trouver la racine d'une équation (f(x) = 0)
  - `minimize` : Trouver le minimum d'une fonction (1D ou multidimensionnelle)
- **Applications** : Machine Learning, finance, physique, modélisation scientifique
- Nécessite toujours une estimation initiale (`x0`) pour démarrer la recherche

### 1.4.1 Ajustement de courbe avec `curve_fit`

In [ ]:
from scipy import optimize

# 1. Génération de données bruitées suivant une fonction cubique
x = np.linspace(0, 2, 100)
y_true = (1/3)*x**3 - (3/5)*x**2 + 2  # Fonction originale
y = y_true + np.random.randn(x.shape[0])/20  # Ajout de bruit

# Visualisation des données bruitées
plt.scatter(x, y, alpha=0.5, label='Données bruitées')
plt.title("Données expérimentales")
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

In [ ]:
# 2. Définition du modèle à ajuster (polynôme de degré 3)
def model(x, a, b, c, d):
    """Modèle cubique avec paramètres a, b, c, d"""
    return a*x**3 + b*x**2 + c*x + d

# 3. Ajustement avec curve_fit (recherche des paramètres optimaux)
params, covariance = optimize.curve_fit(model, x, y)

print("Paramètres optimaux trouvés :")
print(f"  a = {params[0]:.4f}")
print(f"  b = {params[1]:.4f}")
print(f"  c = {params[2]:.4f}")
print(f"  d = {params[3]:.4f}")

In [ ]:
# 4. Visualisation du résultat
plt.scatter(x, y, alpha=0.5, label="Données bruitées")
plt.plot(x, model(x, *params), c='green', lw=2, label="Modèle ajusté")
plt.legend()
plt.title("Ajustement de courbe avec curve_fit")
plt.xlabel('x')
plt.ylabel('y')
plt.show()

### 1.4.2 Trouver la racine d'une équation avec `root`

In [ ]:
from scipy.optimize import root
from math import cos

# Définition de l'équation : x + cos(x) = 0
def equation(x):
    return x + cos(x)

# Recherche de la racine en partant de x0 = 0
solution = root(equation, x0=0)

print(f"Racine trouvée : x = {solution.x[0]:.6f}")
print(f"Convergence réussie : {solution.success}")
print(f"Nombre d'évaluations : {solution.nfev}")

In [ ]:
# Vérification graphique
x_vals = np.linspace(-2, 2, 100)
y_vals = x_vals + np.cos(x_vals)

plt.plot(x_vals, y_vals, label='f(x) = x + cos(x)')
plt.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
plt.scatter(solution.x, 0, color='red', s=100, zorder=5, label=f'Racine x={solution.x[0]:.3f}')
plt.legend()
plt.title("Recherche de racine avec root()")
plt.xlabel('x')
plt.ylabel('f(x)')
plt.grid(True, alpha=0.3)
plt.show()

### 1.4.3 Minimiser une fonction avec `minimize`

In [ ]:
from scipy.optimize import minimize

# === MINIMISATION 1D ===
# Fonction oscillante : f(x) = x² + 15*sin(x)
def f_1d(x):
    return x**2 + 15*np.sin(x)

# Visualisation de la fonction
x_vals = np.linspace(-10, 10, 400)
plt.plot(x_vals, f_1d(x_vals), label='f(x) = x² + 15·sin(x)')
plt.title("Fonction f(x) à minimiser (plusieurs minima locaux)")
plt.xlabel('x')
plt.ylabel('f(x)')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

In [ ]:
# Minimisation à partir d'un point initial x0 = -5
x0 = -5
result = minimize(f_1d, x0=x0)

print(f"Point initial : x₀ = {x0}")
print(f"Minimum local trouvé : x = {result.x[0]:.4f}")
print(f"Valeur minimale : f(x) = {result.fun:.4f}")
print(f"Nombre d'itérations : {result.nit}")

In [ ]:
# Visualisation du résultat
plt.plot(x_vals, f_1d(x_vals), lw=2, label="f(x)")
plt.scatter(x0, f_1d(x0), s=100, marker='+', c='green', linewidths=3, label=f"Point initial ({x0})")
plt.scatter(result.x, result.fun, s=100, c='red', label=f"Minimum trouvé ({result.x[0]:.2f})")
plt.legend()
plt.title("Minimisation locale avec minimize()")
plt.xlabel('x')
plt.ylabel('f(x)')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# === MINIMISATION 2D ===
# Fonction à deux variables : f(x₀, x₁) = sin(x₀) + cos(x₀ + x₁)·cos(x₀)

def f_2d(X):
    x0, x1 = X[0], X[1]
    return np.sin(x0) + np.cos(x0 + x1) * np.cos(x0)

# Création d'une grille pour visualisation
x = np.linspace(-3, 3, 100)
y = np.linspace(-3, 3, 100)
X, Y = np.meshgrid(x, y)
Z = f_2d([X, Y])

# Visualisation avec courbes de niveau
plt.contour(X, Y, Z, 20, cmap='viridis')
plt.colorbar(label='f(x₀, x₁)')
plt.title("Courbes de niveau de f(x₀, x₁)")
plt.xlabel('x₀')
plt.ylabel('x₁')
plt.show()

In [ ]:
# Minimisation 2D à partir de (0, 0)
x0_2d = np.zeros(2)
result_2d = minimize(f_2d, x0=x0_2d)

print(f"Point initial : ({x0_2d[0]}, {x0_2d[1]})")
print(f"Minimum trouvé : ({result_2d.x[0]:.4f}, {result_2d.x[1]:.4f})")
print(f"Valeur minimale : {result_2d.fun:.4f}")

In [ ]:
# Visualisation du résultat 2D
plt.contour(X, Y, Z, 20, cmap='viridis')
plt.colorbar(label='f(x₀, x₁)')
plt.scatter(x0_2d[0], x0_2d[1], marker='+', c='red', s=150, linewidths=3, label="Point initial")
plt.scatter(result_2d.x[0], result_2d.x[1], c='lime', s=150, edgecolors='black', label="Minimum trouvé")
plt.legend()
plt.title("Minimisation 2D avec minimize()")
plt.xlabel('x₀')
plt.ylabel('x₁')
plt.show()

In [ ]:
# === Exemple avec méthode spécifique (BFGS) ===
# Fonction simple : f(x) = x² + x + 2

def f_simple(x):
    return x**2 + x + 2

result_bfgs = minimize(f_simple, x0=0, method='BFGS')

print("=== Minimisation avec méthode BFGS ===")
print(f"Point minimisant : x = {result_bfgs.x[0]:.4f}")
print(f"Valeur minimale : f(x) = {result_bfgs.fun:.4f}")

## 1.5 Interpolation (`scipy.interpolate`)

### 📌 Rappel essentiel

- **Objectif** : Estimer des valeurs inconnues à partir de points connus
- Crée une courbe continue passant entre les points de données existants
- **Fonction principale** : `interp1d(x, y, kind='...')`
- **Types d'interpolation** :
  - `'linear'` : Segments droits (rapide, simple)
  - `'quadratic'` : Courbes quadratiques (plus fluide)
  - `'cubic'` : Splines cubiques (très lisse)
- **Attention** : L'interpolation peut introduire des valeurs irréalistes si mal utilisée

In [ ]:
from scipy.interpolate import interp1d

# Création d'un dataset : 10 points entre 0 et 10
x = np.linspace(0, 10, 10)
y = np.sin(x)  # Fonction sinus

# Visualisation des points d'origine
plt.scatter(x, y, color='blue', s=50, label='Points originaux')
plt.title("Points d'origine (sinus)")
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Création d'une fonction d'interpolation cubique
f_interp = interp1d(x, y, kind='cubic')

# Génération de nouvelles valeurs (50 points au lieu de 10)
new_x = np.linspace(0, 10, 50)
new_y = f_interp(new_x)  # Valeurs interpolées

# Visualisation
plt.scatter(x, y, color='blue', s=50, label='Points originaux')
plt.plot(new_x, new_y, color='red', label='Interpolation cubique')
plt.legend()
plt.title("Interpolation avec scipy.interpolate")
plt.xlabel('x')
plt.ylabel('y')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# === Comparaison des types d'interpolation ===
kinds = ['linear', 'quadratic', 'cubic']
colors = ['red', 'blue', 'green']

plt.scatter(x, y, color='black', s=50, zorder=5, label='Points originaux')

for kind, color in zip(kinds, colors):
    f = interp1d(x, y, kind=kind)
    plt.plot(new_x, f(new_x), label=f'{kind}', color=color)

plt.legend()
plt.title("Comparaison des types d'interpolation")
plt.xlabel('x')
plt.ylabel('y')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# === Cas pratique : compléter des données manquantes ===

# Données avec des points manquants
x_missing = np.array([0, 2, 4, 6, 8, 10])
y_missing = np.sin(x_missing)

# Fonction d'interpolation
f_missing = interp1d(x_missing, y_missing, kind='cubic')

# Remplir les données manquantes
new_x_missing = np.linspace(0, 10, 50)
new_y_missing = f_missing(new_x_missing)

plt.scatter(x_missing, y_missing, color='blue', s=80, zorder=5, label='Données disponibles')
plt.plot(new_x_missing, new_y_missing, color='red', label='Interpolation')
plt.legend()
plt.title("Interpolation pour compléter des données manquantes")
plt.xlabel('x')
plt.ylabel('y')
plt.grid(True, alpha=0.3)
plt.show()

## 1.6 Intégration Numérique (`scipy.integrate`)

### 📌 Rappel essentiel

- **Objectif** : Calculer des intégrales définies et résoudre des équations différentielles
- **Fonctions principales** :
  - `quad()` : Intégrale simple entre deux bornes
  - `dblquad()` : Intégrale double
  - `nquad()` : Intégrales multiples
  - `solve_ivp()` : Résolution d'équations différentielles ordinaires (EDO)
- Retourne la valeur de l'intégrale + estimation de l'erreur numérique

In [ ]:
from scipy import integrate

# === INTÉGRATION SIMPLE ===
# Calculer l'intégrale de sin(x) entre 0 et π
# Résultat attendu : ∫₀^π sin(x)dx = 2

result, error = integrate.quad(np.sin, 0, np.pi)

print("=== Intégration simple ===")
print(f"∫₀^π sin(x) dx = {result:.6f}")
print(f"Erreur estimée : {error:.2e}")

In [ ]:
# Intégrale d'une fonction personnalisée
def f(x):
    return x**2 + 2*x + 1

# ∫₀^1 (x² + 2x + 1) dx
result, error = integrate.quad(f, 0, 1)
print(f"∫₀^1 (x² + 2x + 1) dx = {result:.6f}")

In [ ]:
# === INTÉGRATION DOUBLE ===
# ∬ x·y² dy dx sur le domaine [0,1] × [0,2]

f_double = lambda y, x: x * y**2

result_double, error_double = integrate.dblquad(
    f_double,     # Fonction à intégrer
    0, 1,         # Bornes de x
    lambda x: 0,  # Borne inférieure de y
    lambda x: 2   # Borne supérieure de y
)

print("=== Intégration double ===")
print(f"∬ x·y² dy dx = {result_double:.6f}")

In [ ]:
# === ÉQUATIONS DIFFÉRENTIELLES ORDINAIRES (EDO) ===
# Résoudre dy/dt = -0.5*y avec y(0) = 5
# Solution analytique : y(t) = 5·e^(-0.5t)

from scipy.integrate import solve_ivp

def f_edo(t, y):
    return -0.5 * y

# Résolution entre t=0 et t=10 avec y(0)=5
sol = solve_ivp(f_edo, [0, 10], [5], t_eval=np.linspace(0, 10, 100))

print("=== Équation différentielle ===")
print(f"Valeur initiale : y(0) = 5")
print(f"Valeur finale : y(10) = {sol.y[0, -1]:.4f}")

In [ ]:
# Visualisation de la solution de l'EDO
plt.plot(sol.t, sol.y[0], label='Solution numérique')
plt.plot(sol.t, 5*np.exp(-0.5*sol.t), '--', label='Solution analytique')
plt.xlabel('t')
plt.ylabel('y(t)')
plt.title("Résolution de dy/dt = -0.5·y avec solve_ivp")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 1.7 Algèbre Linéaire (`scipy.linalg`)

### 📌 Rappel essentiel

- **Objectif** : Manipulation de matrices et résolution de systèmes linéaires
- Complète `numpy.linalg` avec des outils plus puissants et précis
- **Fonctions principales** :
  - `solve()` : Résolution de Ax = b
  - `det()` : Calcul du déterminant
  - `inv()` : Calcul de l'inverse
  - `eig()` : Valeurs et vecteurs propres
- **Applications** : PCA, régressions, réseaux de neurones, traitement d'images

In [ ]:
from scipy import linalg

# === RÉSOLUTION DE SYSTÈME LINÉAIRE ===
# Résoudre Ax = b
# 3x + 2y = 8
# x + 4y = 10

A = np.array([[3, 2],
              [1, 4]])

b = np.array([8, 10])

x = linalg.solve(A, b)

print("=== Résolution de système linéaire ===")
print(f"Matrice A :\n{A}")
print(f"Vecteur b : {b}")
print(f"Solution x : {x}")
print(f"Vérification A·x = {A @ x}")

In [ ]:
# === DÉTERMINANT ===
det_A = linalg.det(A)
print(f"Déterminant de A : {det_A:.4f}")

In [ ]:
# === INVERSE ===
inv_A = linalg.inv(A)
print("Inverse de A :")
print(inv_A)

# Vérification : A · A⁻¹ = I
print("\nVérification A · A⁻¹ :")
print(np.round(A @ inv_A, 10))

In [ ]:
# === VALEURS ET VECTEURS PROPRES ===
eigvals, eigvecs = linalg.eig(A)

print("=== Analyse spectrale ===")
print(f"Valeurs propres : {eigvals}")
print(f"\nVecteurs propres :\n{eigvecs}")

In [ ]:
# === DÉCOMPOSITION LU ===
P, L, U = linalg.lu(A)

print("=== Décomposition LU ===")
print(f"P (permutation) :\n{P}")
print(f"\nL (triangulaire inférieure) :\n{L}")
print(f"\nU (triangulaire supérieure) :\n{U}")
print(f"\nVérification P·L·U :\n{P @ L @ U}")

## 📝 Conclusion du Chapitre 1

### Points à retenir :

| Module | Fonction principale | Application |
|--------|---------------------|-------------|
| `scipy.constants` | Constantes physiques/mathématiques | Calculs précis sans erreur de saisie |
| `scipy.optimize` | `curve_fit`, `root`, `minimize` | Ajustement, équations, optimisation |
| `scipy.interpolate` | `interp1d` | Compléter des données manquantes |
| `scipy.integrate` | `quad`, `solve_ivp` | Intégrales, équations différentielles |
| `scipy.linalg` | `solve`, `eig`, `inv` | Systèmes linéaires, algèbre matricielle |

---
# CHAPITRE 2 : Analyse Scientifique et Structures Avancées
---

## 2.1 Fonctions Statistiques (`scipy.stats`)

### 📌 Rappel essentiel

- **Objectif** : Analyse statistique complète des données
- **Fonctions principales** :
  - `describe()` : Résumé statistique complet
  - `tmean()`, `tvar()`, `tstd()` : Moyenne, variance, écart-type
  - `mode()` : Valeur la plus fréquente
- **Distributions** : Plus de 100 distributions (normale, binomiale, Poisson, etc.)
- **Tests statistiques** : t-test, chi-deux, normalité, corrélations
- **Applications** : Data Science, ML, finance, recherche scientifique

In [ ]:
from scipy import stats

# === STATISTIQUES DESCRIPTIVES ===
# Jeu de données : notes d'un groupe d'étudiants
data = np.array([12, 15, 14, 10, 18, 17, 15, 13, 16, 14])

# Résumé complet avec describe
summary = stats.describe(data)

print("=== Statistiques descriptives ===")
print(f"Nombre d'observations : {summary.nobs}")
print(f"Min - Max : {summary.minmax}")
print(f"Moyenne : {summary.mean:.2f}")
print(f"Variance : {summary.variance:.2f}")

In [ ]:
# Fonctions individuelles
print("=== Fonctions individuelles ===")
print(f"Moyenne : {stats.tmean(data):.2f}")
print(f"Médiane : {np.median(data):.2f}")
print(f"Mode : {stats.mode(data, keepdims=True).mode[0]}")
print(f"Variance : {stats.tvar(data):.2f}")
print(f"Écart-type : {stats.tstd(data):.2f}")

In [ ]:
# === DISTRIBUTIONS DE PROBABILITÉS ===
# Distribution normale (loi de Gauss)

mu, sigma = 0, 1  # Paramètres : moyenne = 0, écart-type = 1

x = np.linspace(-5, 5, 1000)
pdf = stats.norm.pdf(x, mu, sigma)  # Probability Density Function

plt.plot(x, pdf, label=f'N(μ={mu}, σ={sigma})')
plt.fill_between(x, pdf, alpha=0.3)
plt.title("Distribution normale (Gaussienne)")
plt.xlabel('x')
plt.ylabel('Densité de probabilité')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Fonctions clés pour les distributions
print("=== Fonctions de distribution ===")

# PDF : Probability Density Function
print(f"PDF à x=0 : {stats.norm.pdf(0):.4f}")

# CDF : Cumulative Distribution Function
print(f"P(X < 1.96) = {stats.norm.cdf(1.96):.4f}")

# Génération d'échantillons aléatoires
samples = stats.norm.rvs(size=5)
print(f"5 échantillons aléatoires : {samples}")

In [ ]:
# === TESTS STATISTIQUES ===
# Test t de Student : la moyenne est-elle significativement différente de 12 ?

t_stat, p_value = stats.ttest_1samp(data, 12)

print("=== Test t de Student ===")
print(f"H₀ : La moyenne = 12")
print(f"Statistique t : {t_stat:.4f}")
print(f"P-value : {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print(f"→ Différence statistiquement significative (α = {alpha})")
else:
    print(f"→ Pas de différence significative (α = {alpha})")

In [ ]:
# Test de normalité (Shapiro-Wilk)
stat_shapiro, p_shapiro = stats.shapiro(data)

print("=== Test de normalité (Shapiro-Wilk) ===")
print(f"Statistique W : {stat_shapiro:.4f}")
print(f"P-value : {p_shapiro:.4f}")

if p_shapiro > 0.05:
    print("→ Les données suivent une distribution normale")
else:
    print("→ Les données ne suivent pas une distribution normale")

In [ ]:
# === CORRÉLATIONS ===
# Corrélation de Pearson entre taille et poids

taille = np.array([150, 160, 165, 170, 175, 180])
poids = np.array([50, 55, 60, 65, 72, 80])

corr, p_value = stats.pearsonr(taille, poids)

print("=== Corrélation de Pearson ===")
print(f"Coefficient de corrélation : {corr:.4f}")
print(f"P-value : {p_value:.6f}")

# Visualisation
plt.scatter(taille, poids)
plt.xlabel('Taille (cm)')
plt.ylabel('Poids (kg)')
plt.title(f'Corrélation taille-poids (r = {corr:.3f})')
plt.grid(True, alpha=0.3)
plt.show()

## 2.2 Matrices Creuses (`scipy.sparse`)

### 📌 Rappel essentiel

- **Objectif** : Stocker efficacement des matrices avec beaucoup de zéros
- **Avantages** : Gain mémoire + performances accrues
- **Formats principaux** :
  - `CSR` (Compressed Sparse Row) : Opérations ligne par ligne
  - `CSC` (Compressed Sparse Column) : Opérations colonne par colonne
  - `LIL`, `DOK`, `COO` : Construction progressive
- **Conversion** : `.toarray()` (sparse → dense), `.tocsr()`, `.tocsc()`
- **Applications** : Graphes, ML (bag-of-words), systèmes de recommandation

In [ ]:
from scipy.sparse import csr_matrix

# === CRÉATION D'UNE MATRICE CSR ===
# Matrice dense avec beaucoup de zéros
dense = np.array([
    [0, 0, 0, 1],
    [5, 0, 0, 0],
    [0, 0, 2, 0]
])

print("Matrice dense :")
print(dense)

# Conversion en matrice CSR
sparse = csr_matrix(dense)

print("\nMatrice creuse (CSR) :")
print(sparse)

In [ ]:
# Interprétation : seuls les éléments non nuls sont stockés
# Format : (ligne, colonne)  valeur
# Exemple : (0, 3) 1 → ligne 0, colonne 3, valeur = 1

# Accès aux données internes
print("=== Structure interne CSR ===")
print(f"Données : {sparse.data}")
print(f"Indices colonnes : {sparse.indices}")
print(f"Pointeurs lignes : {sparse.indptr}")

In [ ]:
# === CONVERSION SPARSE → DENSE ===
dense_back = sparse.toarray()
print("Reconversion en matrice dense :")
print(dense_back)

In [ ]:
# === OPÉRATIONS SUR MATRICES CREUSES ===

mat1 = csr_matrix([[0, 0, 1], [2, 0, 0]])
mat2 = csr_matrix([[1, 0, 0], [0, 0, 3]])

# Addition
result = mat1 + mat2
print("Addition de deux matrices creuses :")
print(result)

# Conversion en dense pour visualisation
print("\nRésultat en format dense :")
print(result.toarray())

In [ ]:
# Somme des éléments
print(f"Somme des éléments : {result.sum()}")

# Multiplication matrice-vecteur
vec = np.array([1, 2, 3])
print(f"Produit matrice-vecteur : {result.dot(vec)}")

In [ ]:
# === CONVERSION ENTRE FORMATS ===

# CSR vers CSC
csc_version = result.tocsc()
print(f"Format CSC : {type(csc_version)}")

# CSR vers COO
coo_version = result.tocoo()
print(f"Format COO : {type(coo_version)}")

## 2.3 Graphes (`scipy.sparse.csgraph`)

### 📌 Rappel essentiel

- **Objectif** : Analyser des structures de graphes (réseaux)
- Un graphe = Nœuds (sommets) + Arêtes (liens)
- **Représentation** : Matrice d'adjacence (stockée en CSR)
- **Fonctions principales** :
  - `connected_components()` : Composantes connexes
  - `dijkstra()` : Plus court chemin (poids positifs)
  - `floyd_warshall()` : Distances entre toutes les paires
  - `bellman_ford()` : Plus court chemin (poids négatifs)
  - `depth_first_order()`, `breadth_first_order()` : Parcours DFS/BFS
- **Applications** : GPS, réseaux sociaux, IA, télécommunications

In [ ]:
from scipy.sparse.csgraph import (
    connected_components, dijkstra, floyd_warshall,
    bellman_ford, depth_first_order, breadth_first_order
)

# === CRÉATION D'UN GRAPHE ===
# Graphe avec 3 sommets : A(0), B(1), C(2)
# A--1--B, A--2--C

graph = np.array([
    [0, 1, 2],  # A: lié à B (poids 1), lié à C (poids 2)
    [1, 0, 0],  # B: lié à A
    [2, 0, 0]   # C: lié à A
])

sparse_graph = csr_matrix(graph)

print("Matrice d'adjacence :")
print(graph)
print("\nReprésentation sparse :")
print(sparse_graph)

In [ ]:
# === COMPOSANTES CONNEXES ===
n_components, labels = connected_components(sparse_graph)

print("=== Composantes connexes ===")
print(f"Nombre de composantes : {n_components}")
print(f"Labels des nœuds : {labels}")

In [ ]:
# === PLUS COURT CHEMIN : DIJKSTRA ===
# Depuis le nœud 0 (A)

distances, predecessors = dijkstra(
    sparse_graph,
    return_predecessors=True,
    indices=0  # Nœud de départ
)

print("=== Dijkstra depuis A ===")
print(f"Distances : {distances}")
print(f"  A→A = {distances[0]}")
print(f"  A→B = {distances[1]}")
print(f"  A→C = {distances[2]}")
print(f"Prédécesseurs : {predecessors}")

In [ ]:
# === FLOYD-WARSHALL : Toutes les paires ===
dist_matrix = floyd_warshall(sparse_graph)

print("=== Floyd-Warshall (toutes les paires) ===")
print("Matrice des distances :")
print(dist_matrix)

In [ ]:
# === BELLMAN-FORD : Poids négatifs ===
graph_neg = np.array([
    [0, -1, 2],
    [1, 0, 0],
    [2, 0, 0]
])

sparse_neg = csr_matrix(graph_neg)
distances_bf = bellman_ford(sparse_neg, indices=0)

print("=== Bellman-Ford (poids négatifs) ===")
print(f"Distances depuis A : {distances_bf}")

In [ ]:
# === PARCOURS DFS ET BFS ===
# Graphe plus complexe pour illustrer
M = np.array([
    [0, 1, 0, 1],
    [1, 1, 1, 1],
    [0, 1, 1, 0],
    [0, 1, 0, 1]
])
mat = csr_matrix(M)

# DFS (Depth-First Search)
order_dfs, pred_dfs = depth_first_order(mat, i_start=1)
print("=== Parcours en profondeur (DFS) ===")
print(f"Ordre de visite : {order_dfs}")
print(f"Prédécesseurs : {pred_dfs}")

# BFS (Breadth-First Search)
order_bfs, pred_bfs = breadth_first_order(mat, i_start=1)
print("\n=== Parcours en largeur (BFS) ===")
print(f"Ordre de visite : {order_bfs}")
print(f"Prédécesseurs : {pred_bfs}")

## 2.4 Algorithmes Spatiaux (`scipy.spatial`)

### 📌 Rappel essentiel

- **Objectif** : Travailler avec des données spatiales (positions dans l'espace)
- **Fonctions principales** :
  - `distance_matrix`, `pdist`, `cdist` : Calcul de distances
  - `KDTree`, `cKDTree` : Recherche rapide de voisins (KNN)
  - `Delaunay` : Triangulation optimale
  - `ConvexHull` : Enveloppe convexe
  - `Voronoi` : Diagramme de Voronoi
- **Applications** : Cartographie, clustering, vision par ordinateur, jeux vidéo

In [ ]:
from scipy import spatial
from scipy.spatial import distance, KDTree, Delaunay, ConvexHull, Voronoi
from scipy.spatial import voronoi_plot_2d

# === CALCUL DE DISTANCES ===
# Deux ensembles de points
points_A = np.array([[0, 0], [1, 1], [2, 2]])
points_B = np.array([[0, 1], [1, 0]])

# Matrice des distances entre A et B
dist_mat = distance.cdist(points_A, points_B, metric='euclidean')

print("=== Matrice des distances ===")
print(f"Points A :\n{points_A}")
print(f"Points B :\n{points_B}")
print(f"\nDistances (A vs B) :\n{dist_mat}")

In [ ]:
# === KDTREE : Recherche rapide de voisins ===
np.random.seed(42)
points = np.random.rand(100, 2)  # 100 points en 2D

# Construction du KDTree
tree = KDTree(points)

# Recherche du point le plus proche de (0.5, 0.5)
query_point = [0.5, 0.5]
dist, idx = tree.query(query_point)

print("=== KDTree ===")
print(f"Point de requête : {query_point}")
print(f"Plus proche voisin : {points[idx]}")
print(f"Distance : {dist:.4f}")

In [ ]:
# Recherche des 3 plus proches voisins
dists, indices = tree.query(query_point, k=3)

print("\n3 plus proches voisins :")
for i, (d, idx) in enumerate(zip(dists, indices)):
    print(f"  {i+1}. Point {points[idx]} - Distance : {d:.4f}")

In [ ]:
# === TRIANGULATION DE DELAUNAY ===
np.random.seed(42)
points_del = np.random.rand(10, 2)

tri = Delaunay(points_del)

plt.triplot(points_del[:, 0], points_del[:, 1], tri.simplices)
plt.scatter(points_del[:, 0], points_del[:, 1], c='red', s=50)
plt.title("Triangulation de Delaunay")
plt.axis('equal')
plt.show()

In [ ]:
# === ENVELOPPE CONVEXE ===
np.random.seed(42)
points_hull = np.random.rand(20, 2)

hull = ConvexHull(points_hull)

plt.scatter(points_hull[:, 0], points_hull[:, 1], c='blue')

# Tracer l'enveloppe convexe
for simplex in hull.simplices:
    plt.plot(points_hull[simplex, 0], points_hull[simplex, 1], 'r-', lw=2)

plt.title("Enveloppe convexe (ConvexHull)")
plt.axis('equal')
plt.show()

print(f"Surface de l'enveloppe : {hull.area:.4f}")

In [ ]:
# === DIAGRAMME DE VORONOI ===
np.random.seed(42)
points_vor = np.random.rand(10, 2)

vor = Voronoi(points_vor)

fig, ax = plt.subplots()
voronoi_plot_2d(vor, ax=ax, show_vertices=False)
ax.scatter(points_vor[:, 0], points_vor[:, 1], c='red', s=50)
ax.set_title("Diagramme de Voronoi")
ax.set_xlim(-0.5, 1.5)
ax.set_ylim(-0.5, 1.5)
plt.show()

## 📝 Conclusion du Chapitre 2

### Points à retenir :

| Module | Fonction principale | Application |
|--------|---------------------|-------------|
| `scipy.stats` | `describe`, `ttest_*`, `pearsonr` | Analyse statistique, tests d'hypothèses |
| `scipy.sparse` | `csr_matrix`, `toarray` | Matrices creuses, gain mémoire |
| `scipy.sparse.csgraph` | `dijkstra`, `floyd_warshall` | Analyse de graphes, plus courts chemins |
| `scipy.spatial` | `KDTree`, `Delaunay`, `ConvexHull` | Données spatiales, voisinage, géométrie |

---
# CHAPITRE 3 : Applications Scientifiques et Préparation aux Projets IA
---

## 3.1 Traitement du Signal (`scipy.signal`)

### 📌 Rappel essentiel

- **Objectif** : Analyser, transformer et filtrer des signaux
- **Types de signaux** : Audio, capteurs, séries temporelles, ECG, EEG...
- **Fonctions principales** :
  - `signal.detrend()` : Supprimer une tendance linéaire
  - `fftpack.fft()` : Transformée de Fourier (analyse fréquentielle)
  - `fftpack.ifft()` : Transformée inverse (reconstruction)
  - `fftpack.fftfreq()` : Générer les fréquences correspondantes
- **Applications** : Nettoyage audio, détection d'anomalies, filtrage de données

In [ ]:
from scipy import signal
from scipy import fftpack

# === SUPPRESSION DE TENDANCE ===
# Signal avec une tendance linéaire + oscillations

x = np.linspace(0, 20, 100)
y = x + 4 * np.sin(x) + np.random.randn(x.shape[0])  # Tendance linéaire + sinus + bruit

plt.plot(x, y, label="Signal original")
plt.title("Signal avec tendance linéaire")
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Suppression de la tendance
y_detrended = signal.detrend(y)

plt.plot(x, y, label="Original", alpha=0.7)
plt.plot(x, y_detrended, label="Detrend", lw=2)
plt.legend()
plt.title("Avant et après suppression de la tendance")
plt.xlabel('x')
plt.ylabel('y')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# === TRANSFORMÉE DE FOURIER (FFT) ===
# Signal composé de plusieurs sinusoïdes + bruit

x = np.linspace(0, 30, 1000)
y = 3*np.sin(x) + 2*np.sin(5*x) + np.sin(10*x) + np.random.random(x.shape[0])*10

plt.plot(x, y)
plt.title("Signal bruité (3 fréquences + bruit)")
plt.xlabel('Temps')
plt.ylabel('Amplitude')
plt.show()

In [ ]:
# Étape 1 : Calcul de la FFT
fourier = fftpack.fft(y)

# Module de la FFT (amplitudes)
power = np.abs(fourier)

# Fréquences associées
frequencies = fftpack.fftfreq(y.size)

# Visualisation du spectre
plt.plot(np.abs(frequencies), power)
plt.title("Spectre de fréquences (FFT)")
plt.xlabel('Fréquence')
plt.ylabel('Amplitude')
plt.xlim(0, 0.1)  # Zoom sur les basses fréquences
plt.show()

In [ ]:
# Étape 2 : Filtrage des fréquences faibles (suppression du bruit)
fourier_filtered = fourier.copy()
fourier_filtered[power < 400] = 0  # Supprime les amplitudes faibles

plt.plot(np.abs(frequencies), np.abs(fourier_filtered))
plt.title("Spectre après filtrage")
plt.xlabel('Fréquence')
plt.ylabel('Amplitude')
plt.xlim(0, 0.1)
plt.show()

In [ ]:
# Étape 3 : Reconstruction du signal filtré (IFFT)
filtered_signal = fftpack.ifft(fourier_filtered)

plt.figure(figsize=(12, 5))
plt.plot(x, y, lw=0.5, alpha=0.7, label='Signal original')
plt.plot(x, filtered_signal.real, lw=2, label='Signal filtré')
plt.legend()
plt.title("Signal avant et après filtrage FFT")
plt.xlabel('Temps')
plt.ylabel('Amplitude')
plt.show()

## 3.2 Traitement d'Images (`scipy.ndimage`)

### 📌 Rappel essentiel

- **Objectif** : Analyser et transformer des images (2D, 3D, N-D)
- **Morphologie mathématique** :
  - `binary_erosion` : Supprime les pixels isolés
  - `binary_dilation` : Agrandit les formes
  - `binary_opening` : Érosion + Dilatation (nettoie le bruit)
  - `binary_closing` : Dilatation + Érosion (comble les trous)
- **Autres opérations** :
  - `gaussian_filter` : Lissage gaussien
  - `rotate`, `zoom` : Transformations géométriques
- **Applications** : Médical (IRM), industrie, vision par ordinateur

In [ ]:
from scipy import ndimage

# === CRÉATION D'UNE IMAGE AVEC BRUIT ===
np.random.seed(0)

# Image 32x32 (noir)
image = np.zeros((32, 32))

# Ajout d'un carré blanc au centre
image[10:-10, 10:-10] = 1

# Ajout de bruit (pixels parasites)
noise_x = np.random.randint(0, 32, 30)
noise_y = np.random.randint(0, 32, 30)
image[noise_x, noise_y] = 1

plt.imshow(image, cmap='gray')
plt.title("Image originale avec artefacts")
plt.axis('off')
plt.show()

In [ ]:
# === BINARY OPENING : Nettoyage du bruit ===
# Opening = Érosion puis Dilatation

clean_image = ndimage.binary_opening(image)

plt.imshow(clean_image, cmap='gray')
plt.title("Après binary_opening (bruit supprimé)")
plt.axis('off')
plt.show()

In [ ]:
# === Comparaison avant/après ===
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

axes[0].imshow(image, cmap='gray')
axes[0].set_title("Avant (avec bruit)")
axes[0].axis('off')

axes[1].imshow(clean_image, cmap='gray')
axes[1].set_title("Après (binary_opening)")
axes[1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# === AUTRES OPÉRATIONS MORPHOLOGIQUES ===

# Binary erosion (érosion seule)
eroded = ndimage.binary_erosion(image)

# Binary dilation (dilatation seule)
dilated = ndimage.binary_dilation(image)

# Binary closing (fermeture : dilatation puis érosion)
closed = ndimage.binary_closing(image)

fig, axes = plt.subplots(2, 2, figsize=(10, 10))

axes[0, 0].imshow(image, cmap='gray')
axes[0, 0].set_title("Original")
axes[0, 0].axis('off')

axes[0, 1].imshow(eroded, cmap='gray')
axes[0, 1].set_title("Érosion")
axes[0, 1].axis('off')

axes[1, 0].imshow(dilated, cmap='gray')
axes[1, 0].set_title("Dilatation")
axes[1, 0].axis('off')

axes[1, 1].imshow(closed, cmap='gray')
axes[1, 1].set_title("Fermeture (closing)")
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# === FILTRE GAUSSIEN ===
# Lissage progressif de l'image

smoothed = ndimage.gaussian_filter(image.astype(float), sigma=1.5)

fig, axes = plt.subplots(1, 2, figsize=(10, 4))

axes[0].imshow(image, cmap='gray')
axes[0].set_title("Original")
axes[0].axis('off')

axes[1].imshow(smoothed, cmap='gray')
axes[1].set_title("Filtre Gaussien (σ=1.5)")
axes[1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# === ROTATION ET ZOOM ===

# Rotation de 45 degrés
rotated = ndimage.rotate(image, angle=45, reshape=False)

# Zoom x2
zoomed = ndimage.zoom(image, zoom=2)

fig, axes = plt.subplots(1, 3, figsize=(12, 4))

axes[0].imshow(image, cmap='gray')
axes[0].set_title("Original")
axes[0].axis('off')

axes[1].imshow(rotated, cmap='gray')
axes[1].set_title("Rotation 45°")
axes[1].axis('off')

axes[2].imshow(zoomed, cmap='gray')
axes[2].set_title("Zoom x2")
axes[2].axis('off')

plt.tight_layout()
plt.show()

## 3.3 Datasets (`scipy.datasets`)

### 📌 Rappel essentiel

- **Objectif** : Accéder à des jeux de données intégrés pour l'apprentissage
- **Datasets disponibles** (via sklearn, compatible avec SciPy) :
  - `load_iris()` : Classification de fleurs (150 échantillons, 3 classes)
  - `load_digits()` : Reconnaissance de chiffres manuscrits
  - `load_diabetes()` : Données médicales
  - `load_wine()` : Classification de vins
- **Structure** : `data` (features), `target` (labels), `feature_names`, `target_names`
- **Applications** : Test d'algorithmes, prototypage, enseignement

In [ ]:
# Note : scipy.datasets est limité, on utilise sklearn.datasets
# qui est l'approche standard pour les datasets scientifiques
from sklearn import datasets

# === DATASET IRIS ===
iris = datasets.load_iris()

print("=== Dataset Iris ===")
print(f"Clés disponibles : {iris.keys()}")
print(f"\nDimensions des données : {iris.data.shape}")
print(f"Nombre de classes : {len(iris.target_names)}")
print(f"Classes : {iris.target_names}")
print(f"Features : {iris.feature_names}")

In [ ]:
# Conversion en DataFrame pour une meilleure visualisation
import pandas as pd

iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['species'] = [iris.target_names[t] for t in iris.target]

print("5 premières lignes du dataset Iris :")
iris_df.head()

In [ ]:
# Visualisation du dataset Iris
plt.scatter(iris.data[:, 2], iris.data[:, 3], c=iris.target, cmap='viridis')
plt.xlabel('Longueur des pétales (cm)')
plt.ylabel('Largeur des pétales (cm)')
plt.title('Dataset Iris - Pétales')
plt.colorbar(label='Espèce')
plt.show()

In [ ]:
# === DATASET DIGITS ===
digits = datasets.load_digits()

print("=== Dataset Digits ===")
print(f"Dimensions des données : {digits.data.shape}")
print(f"Dimensions des images : {digits.images.shape}")
print(f"Classes : {digits.target_names}")

In [ ]:
# Affichage de quelques chiffres
fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(digits.images[i], cmap='gray')
    ax.set_title(f"Label: {digits.target[i]}")
    ax.axis('off')

plt.suptitle("Dataset Digits - Exemples")
plt.tight_layout()
plt.show()

In [ ]:
# === DATASET WINE ===
wine = datasets.load_wine()

print("=== Dataset Wine ===")
print(f"Dimensions : {wine.data.shape}")
print(f"Classes : {wine.target_names}")
print(f"Features : {wine.feature_names[:5]}...")  # 5 premières

In [ ]:
# === DATASET DIABETES ===
diabetes = datasets.load_diabetes()

print("=== Dataset Diabetes ===")
print(f"Dimensions : {diabetes.data.shape}")
print(f"Features : {diabetes.feature_names}")
print(f"\nPremières valeurs cibles : {diabetes.target[:5]}")

In [ ]:
# === Tableau récapitulatif des datasets ===
print("\n=== Récapitulatif des datasets ===")
print(f"{'Dataset':<15} {'Échantillons':<15} {'Features':<15} {'Type'}")
print("-" * 60)
print(f"{'Iris':<15} {iris.data.shape[0]:<15} {iris.data.shape[1]:<15} {'Classification'}")
print(f"{'Digits':<15} {digits.data.shape[0]:<15} {digits.data.shape[1]:<15} {'Classification'}")
print(f"{'Wine':<15} {wine.data.shape[0]:<15} {wine.data.shape[1]:<15} {'Classification'}")
print(f"{'Diabetes':<15} {diabetes.data.shape[0]:<15} {diabetes.data.shape[1]:<15} {'Régression'}")

## 📝 Conclusion du Chapitre 3

### Points à retenir :

| Module | Fonction principale | Application |
|--------|---------------------|-------------|
| `scipy.signal` | `detrend`, `fft`, `ifft` | Filtrage, analyse fréquentielle |
| `scipy.ndimage` | `binary_opening`, `gaussian_filter` | Traitement d'images, morphologie |
| `sklearn.datasets` | `load_iris`, `load_digits` | Datasets pour ML et prototypage |

---
# 🎯 Conclusion Générale
---

Ce notebook a couvert l'ensemble de la bibliothèque **SciPy** :

### Chapitre 1 - Fondamentaux
- Constantes scientifiques (`scipy.constants`)
- Optimisation et ajustement (`scipy.optimize`)
- Interpolation (`scipy.interpolate`)
- Intégration numérique (`scipy.integrate`)
- Algèbre linéaire (`scipy.linalg`)

### Chapitre 2 - Structures avancées
- Statistiques (`scipy.stats`)
- Matrices creuses (`scipy.sparse`)
- Graphes (`scipy.sparse.csgraph`)
- Algorithmes spatiaux (`scipy.spatial`)

### Chapitre 3 - Applications
- Traitement du signal (`scipy.signal`, `scipy.fftpack`)
- Traitement d'images (`scipy.ndimage`)
- Datasets pour le Machine Learning

---

**SciPy** constitue le socle fondamental pour :
- **Pandas** (analyse tabulaire)
- **Scikit-Learn** (apprentissage automatique)
- **TensorFlow/PyTorch** (deep learning)

La maîtrise de SciPy vous prépare aux modules avancés de Data Science et d'Intelligence Artificielle.

## Ressources
- https://docs.scipy.org/doc/scipy/

> **Prochaine séance :**
> * Pandas —

Bon courage à tous, et surtout : codez, cassez, réparez, recommencez.  

C’est comme ça qu’on devient bon en **Data Science**.

À la semaine prochaine inchae ALLAH !

<br>
<hr>
<div style="font-size:14px; line-height:1.5;">
<strong style="font-size:16px;">Y. EL ALLIOUI</strong><br>
<span style="color:#555;">FPK – USMS</span><br>
<a href="mailto:y.elallioui@usms.ma" style="color:#2c3e50; text-decoration:none;">
y.elallioui@usms.ma
</a>
</div>